In [14]:
import lineRenderWorldSetup as LR
import cv2
import numpy as np
from mayavi import mlab
import matplotlib.pyplot as plt
from glob import glob
from natsort import natsorted
import os
import _3dTo2dSetup as D3

In [16]:
#path to image folder and load 3d model
path = r"data\segment"
img_paths = natsorted(glob(os.path.join(path, "*.tiff"))) #.png
dim = 735
box_dim = LR.BOX_DIM
flip = True #False
#load 3D model
path_real = r"solid_400001-segmentation-solid_400001-label.nrrd"
real_box = D3.get3dFigure(box_dim, path_real)

angles = LR.ANGLES
print(img_paths)

['data\\segment\\I0_021.tiff', 'data\\segment\\I1_016.tiff', 'data\\segment\\I3_023.tiff', 'data\\segment\\I4_021.tiff', 'data\\segment\\I6_031.tiff', 'data\\segment\\I7_023.tiff', 'data\\segment\\I8_021.tiff']


In [17]:
prim_rots = []
secd_rots = []
imgs = []

for i in range(len(angles)):
  p = img_paths[i]
  img = cv2.imread(p, 0)
  f = os.path.basename(p)
  imgs.append(img)

  prim, secd = angles[i]
  prim_rots.append(prim)
  secd_rots.append(secd)


In [18]:
#create back-projection for each image
boxes = []

for i in imgs:
  line_coords = LR.createCoordsForLoadedImageFrom3d(i, flip, dim)
  world_coords = LR.performBresenham3D(
      line_coords, LR.DETECTOR_TO_PATIENT, LR.SRC_TO_PATIENT, box_dim)
  box = LR.savePointsWithinSquare(world_coords, box_dim)
  boxes.append(box)
  print("produced box")


produced box
produced box
produced box
produced box
produced box
produced box
produced box


In [19]:
#add rotation to each box 
for i in range(len(boxes)):
  box_rot = LR.rotatePrimSecond(boxes[i], prim_rots[i], secd_rots[i])
  box_rot = np.where(box_rot > 0.5, 1, 0)
  boxes[i] = box_rot 
  print("Rotated box ", i)

Rotated box  0
Rotated box  1
Rotated box  2
Rotated box  3
Rotated box  4
Rotated box  5
Rotated box  6


In [12]:
# Save box projections for each image as numpy arrays
for i in range(len(boxes)):
  p = img_paths[i]
  f = os.path.basename(p)
  angle = str(prim_rots[i]) + "_" + str(secd_rots[i]) 
  np.save(r"model_uncertainty\docAnglesSeg\S" + angle, boxes[i])